# Denoising using principal component analysis (PCA)

This iPython notebook will conduct PCA to denoise data. Denoising experimental  
data may improve visulization, when signal-to-noise ratio is low.

The data are expected to contain $x$-values in the first column and the  
$y$-values in the second column. Header as well as additional columns can be  
handled.

The data files should be names alphanumerically to be loaded in the right order.

The explained variance ratio (evr) can be used to set the level of the filtering  
for the denoising.

A plot of the explained variance ratio as functions of the number of components  
used during the PCA will be saved together with overview plots of the data,  
the pca-reconstructed (denoised) data, and the difference between the two.

Imports.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from diffpy.utils.parsers.loaddata import loadData

Ensuring that data files are in place.

In [ ]:
data_path = Path.cwd() / "data"
if not data_path.exists():
    data_path.mkdir()
    sys.exit(f"\n{80*'-'}\nA folder called '{data_path.name}' has been created."
             f"\nPlease place your data files there and rerun the cell."
             f"\n{80*'-'}")
data_files = list(data_path.glob("*.*"))
if len(data_files) == 0:
    sys.exit(f"\n{80*'-'}\nNo files were found in the '{data_path.name}' "
             f"folder.\nPlease place your data files there and rerun the cell."
             f"\n{80*'-'}")
s = f"The following files were found in the '{data_path.name}' folder:\n"
for i, e in enumerate(data_files):
    s += f"{i}\t{e.name}\n"
print(f"\n{80*'-'}\n{s}{80*'-'}")

Loading data from files.

In [ ]:
d = {}
print(f"\n{80*'-'}\nLoading data...")
for i, e in enumerate(data_files):
    print(f"\t{i}\t{e.name}")
    data = loadData(e)
    x, y = data[:, 0], data[:, 1]
    d[i] = dict(path=e, x=x, y=y)
    if i == 0:
        yarray = y
    else:
        yarray = np.column_stack((yarray, y))
print(f"Done loading data.\n{80*'-'}\n"
      f"xmin = {np.amin(x)}, xmax = {np.amax(x)}\n{80*'-'}\n"
      f"shape of stacked y-array: {yarray.shape}\n{80*'-'}")

Function to get index of value in array.

In [ ]:
def get_idx(array, value):
    for i, e in enumerate(array):
        if e >= value:
            break
    
    return i

State minimum and maximum $x$-value to conduct analysis for.

In [ ]:
xmin, xmax = 1, 30

Shaping $y$-array to conduct pca for.

In [ ]:
idx_min, idx_max = get_idx(x, xmin), get_idx(x, xmax)
X = yarray[idx_min:idx_max, :].T
print(f"\n{80*'-'}\nshape of X (shaped yarray): {X.shape}\n{80*'-'}")

Plot function to plot explained variance ratio.

In [ ]:
def plot_evr(evr, plot_paths):
    x = np.arange(1, len(evr) + 1)
    xmin, xmax = np.amin(x), np.amax(x)
    xrange = xmax - xmin
    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    fig, axs = plt.subplots(figsize=(14, 6), ncols=2)
    axs[0].plot(x, evr, "-o", c=colors[0])
    axs[0].set_ylabel(r"evr", fontsize=20)
    axs[1].plot(x, np.cumsum(evr), "-o", c=colors[1], label="cumulated")
    axs[1].legend(framealpha=0, fontsize=14)
    for ax in axs:
        ax.tick_params(axis="both", which="both", labelsize=14)
        ax.set_xlim(xmin - 0.01 * xrange, xmax + 0.01 * xrange)
        ax.minorticks_on()
    fig.supxlabel(r"$n$", fontsize=20)
    for p in plot_paths:
        plt.savefig(p / f"evr.{p.name}", bbox_inches="tight", dpi=300)
    plt.show()

    return None

Plotting explained variance ratio as a function of the number of components.

In [ ]:
pca = PCA(n_components=10)
pca.fit(X)
plot_folders = ["png"]
plot_paths = [Path.cwd() / folder for folder in plot_folders]
for p in plot_paths:
    if not p.exists():
        p.mkdir()
print(f"\n{80*'-'}\nExplained variance ratio as a function of number of "
      f"components:")
plot_evr(pca.explained_variance_ratio_, plot_paths)
print(f"{80*'-'}")

Function definition to denoise using pca.

In [ ]:
def pca_denoising(X, n):
    pca_filter = PCA(n)
    pca_filter.fit(X)
    X_transform = pca_filter.transform(X)
    X_inverse_transform = pca_filter.inverse_transform(X_transform)
    d = dict(transform=X_transform,
             inverse_transform=X_inverse_transform,
             evr=pca_filter.explained_variance_ratio_,
             )

    return d

Number of components to user for pca.

In [ ]:
n = 4

Denoising using pca.

In [ ]:
d_pca = pca_denoising(X, n)
print(f"{80*'-'}\nnumber of components: {n}\n"
      f"cumulated explained variance ratio: {np.sum(d_pca['evr'])}\n"
      f"transform shape: {d_pca['transform'].shape}\n"
      f"inverse transform shape: {d_pca['inverse_transform'].shape}\n{80*'-'}")

Plot function for overview plot.

In [ ]:
def plot_overview(X, xmin, xmax, cmap, name, plot_paths):
    fig, ax = plt.subplots(figsize=(12, 8))
    im = ax.imshow(X.T, 
                   aspect="auto", 
                   cmap="seismic",
                   extent=(0, X.T.shape[1], xmax, xmin),
                   interpolation="None",
                   )
    ax.xaxis.set_label_position("top")
    ax.set_xlabel("index", fontsize=20)
    ax.set_ylabel(r"$x$", fontsize=20)
    ax.tick_params(axis="x",
                   which="both",
                   bottom=True, 
                   top=True, 
                   labelbottom=False, 
                   labeltop=True,
                   direction="in",
                   labelsize=14,
                   )
    ax.tick_params(axis="y",
                   which="both",
                   left=True,
                   right=True,
                   labelleft=True,
                   labelright=False,
                   direction="in",
                   labelsize=14,
                   )
    cbar = plt.colorbar(im)
    cbar.set_label(r"$y$")
    ax.minorticks_on()
    for p in plot_paths:
        plt.savefig(p / f"overview_{name}.{p.name}", 
                    bbox_inches="tight", 
                    dpi=300,
                    )
    plt.show()

    return None

Colormap to use for overview plots.

In [ ]:
# cmap = "viridis"
cmap = "seismic"

Plotting data, pca-reconstructed data, and difference.

In [ ]:
print(f"\n{80*'-'}\ndata:")
plot_overview(X, xmin, xmax, cmap, "data", plot_paths)
print(f"pca-reconstructed data:")
plot_overview(d_pca["inverse_transform"], xmin, xmax, cmap, "pca-recon", plot_paths)
print(f"difference:")
plot_overview(X - d_pca["inverse_transform"], xmin, xmax, cmap, "diff", plot_paths)
print(f"Done.\n{80*'-'}")

In [ ]:
x_shaped = x[idx_min:idx_max]
data_pcarecon_path = Path.cwd() / "data_pca-recon"
if not data_pcarecon_path.exists():
    data_pcarecon_path.mkdir()
print(f"\n{80*'-'}\nwriting pca-reconstrucred data to files...")
for i, k in enumerate(d.keys()):
    name = f"{d[k]['path'].stem}_pca-recon{d[k]['path'].suffix}"
    print(f"\t{name}")
    np.savetxt(data_pcarecon_path / name,
               np.column_stack((x_shaped, d_pca["inverse_transform"][i, :])),
               delimiter="\t",
               encoding="utf8",
               )
print(f"Done. Please see the '{data_pcarecon_path.name}' folder.\n{80*'-'}")